In [2]:
sc.stop()

In [3]:
from pyspark import SparkContext, SparkConf

# 예제 2-4
class PassingFunctionSample():

    def add1(self, i):
        return i + 1

    def runMapSample1(self, sc):
        rdd1 = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        # rdd2 = rdd1.map(self.add1)
        rdd2 = rdd1.map(add2)
        print(", ".join(str(i) for i in rdd2.collect()))


if __name__ == "__main__":

    def add2(i):
        return i + 1

    conf = SparkConf()
    sc = SparkContext(master="local[*]", appName="PassingFunctionSample", conf=conf)

    obj = PassingFunctionSample()
    obj.runMapSample1(sc)

    sc.stop()

2, 3, 4, 5, 6, 7, 8, 9, 10, 11


### SparkContext

#### 1. SC와 RDD 생성
* 스파크컨텍스트는 스파크 애플리케이션과 클러스터의 연결을 관리하는 객채.
* RDD를 비롯한 스파크에서 사용하는 주요 객채는 스파크컨텍스트를 이용해 생성 가능.

In [4]:
sc.stop()

In [5]:
sc = SparkContext(master = "local", appName="RDDCreateTest", conf = conf)

In [6]:
rdd1 = sc.parallelize(["a", "b", "c", "d", "e"])

In [7]:
rdd2 = sc.parallelize("/usr/local/Cellar/apache-spark/2.3.2/README.md") # 외부 파일을 이용한 RDD 파일 생성

In [8]:
rdd3 = sc.parallelize(range(1,1000), 10) # 파티션의 갯수를 정할 수 있음 (10)

In [9]:
rdd3

PythonRDD[3] at RDD at PythonRDD.scala:52

#### 2. RDD 기본 액션
* RDD가 제공하는 연산은 트랜스포메이션과 액션 연산으로 나뉨.
* 두 연산의 구분은 수행결과가 RDD인지 아닌지로 판단 가능.

In [10]:
# collect
rdd4 = sc.parallelize(range(1,11))
result = rdd4.collect() # RDD데이터를 array로 돌려줌, 액션 연산
print(result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [11]:
# count
result = rdd4.count()
print(result)

10


#### 3. RDD 트랜스포메이션
* 새로운 RDD를 생성하는 연산
* 맵(map) 연산 : 요소 간의 사상을 정의한 함수를 RDD에 속하는 모든 요소에 적용해 새로운 RDD를 생성.
* 그룹화 연산 : 특정 조건에 따라 요소를 그룹화 & 함수 적용
* 집합 연산 : RDD에 포함된 요소를 하나의 집합으로 간주할때 사용
* 파티션 연산 : RDD의 파티션 개수 조정
* 필터와 정렬 연산 : 특정 조건을 만족하는 요소만 선택하거나 각 요소를 정해진 기준에 따라 정렬

##### map 연산

In [12]:
# map
rdd1 = sc.parallelize(range(0,6))
rdd2 = rdd1.map(lambda v : v + 1)
print(rdd2.collect())

[1, 2, 3, 4, 5, 6]


In [13]:
print(rdd1.collect())

[0, 1, 2, 3, 4, 5]


In [16]:
#flatMap
rdd1 = sc.parallelize(["apple,orrange","grape,apple,mango","blueberry,tomato,orange"])
rdd2 = rdd1.flatMap(lambda s : s.split(","))
print(rdd2.collect())

['apple', 'orrange', 'grape', 'apple', 'mango', 'blueberry', 'tomato', 'orange']


In [20]:
# 기본 rdd
rdd1 = sc.parallelize(["apple,orrange","grape,apple,mango","blueberry,tomato,orange"])
print(rdd1.collect())

['apple,orrange', 'grape,apple,mango', 'blueberry,tomato,orange']


In [21]:
# Map 비교
rdd3 = rdd1.map(lambda s : s.split(","))
print(rdd3.collect())

[['apple', 'orrange'], ['grape', 'apple', 'mango'], ['blueberry', 'tomato', 'orange']]


In [26]:
# mapPartitions - error
rdd1 = sc.parallelize(range(1,11))
rdd2 = rdd1.mapPartitions(increase)
print(rdd2.collect())

def increase(numbers):
    print("DB 연결 !!!")
    return (i + 1 for i in number)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 11, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/rdd.py", line 2440, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/rdd.py", line 350, in func
    return f(iterator)
  File "<ipython-input-24-018d63b67667>

In [29]:
# mapValues
rdd1 = sc.parallelize(["a","b","c"])
rdd2 = rdd1.map(lambda v : (v, 1))
rdd3 = rdd2.mapValues(lambda i : i + 1)

print(rdd2.collect())
print(rdd3.collect())


[('a', 1), ('b', 1), ('c', 1)]
[('a', 2), ('b', 2), ('c', 2)]


In [30]:
# flatMapValues
rdd1 = sc.parallelize([(1, "a,b"),(2,"a,c"),(1,"d,e")])
rdd2 = rdd1.flatMapValues(lambda s : s.split(","))
print(rdd2.collect())

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'c'), (1, 'd'), (1, 'e')]


##### 그룹 연산 

In [31]:
# zip
rdd1 = sc.parallelize(["a","b","c"])
rdd2 = sc.parallelize({1, 2, 3})
result = rdd1.zip(rdd2)
print(result.collect())

[('a', 1), ('b', 2), ('c', 3)]


In [33]:
# groupBy
rdd1 = sc.parallelize(range(1, 11))
rdd2 = rdd1.groupBy(lambda v : "even" if v % 2 == 0 else "odd")
for x in rdd2.collect():
    print(x[0], list(x[1]))

odd [1, 3, 5, 7, 9]
even [2, 4, 6, 8, 10]


In [34]:
# cogroup
rdd1 = sc.parallelize([("k1","v1"),("k2","v2"),("k1","v3")])
rdd2 = sc.parallelize([("k1","v4")])
result = rdd1.cogroup(rdd2)
for x in result.collect():
    print(x[0], list(x[1][0]), list(x[1][1]))

k2 ['v2'] []
k1 ['v1', 'v3'] ['v4']


#####  집합 관련 연산

In [36]:
# distnct
rdd = sc.parallelize([1, 2, 3, 1, 2, 3, 1, 2, 3])
result = rdd.distinct()
print(result.collect())

[1, 2, 3]


In [37]:
# cartesian
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize(["a","b","c"])
result = rdd1.cartesian(rdd2)
print(result.collect())

[(1, 'a'), (1, 'b'), (1, 'c'), (2, 'a'), (2, 'b'), (2, 'c'), (3, 'a'), (3, 'b'), (3, 'c')]


In [39]:
# union
rdd1 = sc.parallelize(["a","b","c"])
rdd2 = sc.parallelize(["d","e","f"])
result = rdd1.union(rdd2)
print(result.collect())

['a', 'b', 'c', 'd', 'e', 'f']


In [40]:
# intersection
rdd1 = sc.parallelize(["a","a","b","c"])
rdd2 = sc.parallelize(["a","a","c","c"])
result = rdd1.intersection(rdd2)
print(result.collect())

['c', 'a']


In [42]:
# join
rdd1 = sc.parallelize(["a","b","c","d","e"]).map(lambda v : (v, 1))
rdd2 = sc.parallelize(["b","c"]).map(lambda v : (v, 2))
result = rdd1.join(rdd2)
print(result.collect())


[('b', (1, 2)), ('c', (1, 2))]


In [46]:
# leftOuterJoin & rightOuterJoin
# 결과가 책과는 조금 다름
rdd1 = sc.parallelize(["a","b","c"]).map(lambda v : (v, 1))
rdd2 = sc.parallelize(["b","c"]).map(lambda v : (v, 2))
result1 = rdd1.leftOuterJoin(rdd2)
result2 = rdd1.rightOuterJoin(rdd2)
print("Left: %s" % result1.collect())
print("Right: %s" % result2.collect())

Left: [('b', (1, 2)), ('c', (1, 2)), ('a', (1, None))]
Right: [('b', (1, 2)), ('c', (1, 2))]


In [47]:
# subtractByKey
rdd1 = sc.parallelize(["a","b"]).map(lambda v: (v, 1))
rdd2 = sc.parallelize(["b"]).map(lambda v: (v, 1))
result = rdd1.subtractByKey(rdd2)
print(result.collect())

[('a', 1)]


#####  집계와 관련된 연산들

In [15]:
#reduceByKey

rdd = sc.parallelize(["a","b","b"]).map(lambda v : (v, 1))
result = rdd.reduceByKey(lambda v1, v2: v1 + v2)
print(result.collect())

[('a', 1), ('b', 2)]


In [17]:
rdd.collect()

[('a', 1), ('b', 1), ('b', 1)]

In [20]:
# combineByKey

class Record:

    def __init__(self, amount, number=1):
        self.amount = amount
        self.number = number
        
    def addAmt(self, amount):
        return Record(self.amount + amount, self.number + 1)
    
    def __add__(self, other):
        amount = self.amount + other.amount
        number = self.number + other.number 
        return Record(amount, number)
        
    def __str__(self):
        return "avg:" + str(self.amount / self.number)

    def __repr__(self):
        return 'Record(%r, %r)' % (self.amount, self.number)

    
rdd = sc.parallelize([("Math", 100),("Eng", 80), ("Math", 50),("Eng", 70), ("Eng", 90)])
rdd.collect()

[('Math', 100), ('Eng', 80), ('Math', 50), ('Eng', 70), ('Eng', 90)]

In [19]:

result = rdd.combineByKey(lambda v: createCombiner(v), lambda c, v: mergeValue(c,v), lambda c1, c2 : mergeCombiners(c1, c2))
print(str(result.collectAsMap()))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/rdd.py", line 2440, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/rdd.py", line 350, in func
    return f(iterator)
  File "/usr/local/Cellar/apache-spark/2.3

##### 필터와 정렬 연산

In [24]:
# fillter - random sample 할때 사용할 수 있을 듯
rdd1 = sc.parallelize(range(1, 20))
rdd2 = rdd1.filter(lambda i : i % 2 == 0)
print(rdd2.collect())

[2, 4, 6, 8, 10, 12, 14, 16, 18]


In [26]:
# sortBykey - 키값을 기준으로 sort
rdd = sc.parallelize([("q", 1),("z", 1),("a", 1),("c", 3), ("y", 22), ("b", 1)])
result = rdd.sortByKey()
print(result.collect())

[('a', 1), ('b', 1), ('c', 3), ('q', 1), ('y', 22), ('z', 1)]


In [30]:
# keys & values
rdd = sc.parallelize([("q", 1),("z", 1),("a", 1),("c", 3), ("y", 22), ("b", 1)])
result1 = rdd.keys()
result2 = rdd.values()
print(result1.collect())
print(result2.collect())

['q', 'z', 'a', 'c', 'y', 'b']
[1, 1, 1, 3, 22, 1]


In [32]:
# sample
rdd = sc.parallelize(range(1,20))
result1 = rdd.sample(False,0.5)
result2 = rdd.sample(True,1.5)
result3 = rdd.sample(False,1.2) # 안되야되는데 됨...
print(result1.collect())
print(result2.collect())
print(result3.collect())

[1, 4, 5, 6, 8, 9, 11, 16, 17, 18]
[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 6, 7, 7, 9, 10, 13, 13, 14, 14, 15, 16, 16, 16, 16, 16, 17, 17, 17, 19]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
